### Interactive Layer Clipping

## Step 1: Define your AOI

Draw a rectangle on the map using the rectangle control. This will be used as the bounding box of the clipped raster.

## Step 2: Define your target SRS

Provide your target SRS as an EPSG code.  The identified name of the EPSG code is provided for your reference.

You must also provide a cell size in your target coordinate system's units.

## Step 3: Click the `submit` button to start clipping

Logging will be printed below the map.

In [1]:
import os
import logging
import sys
from urllib.parse import parse_qs

import ipywidgets
from IPython import get_ipython
import IPython.display

LOGGER = logging.getLogger(__name__)

# Check the query parameters if we're in voila
if (os.environ.get('SERVER_SOFTWARE', False) and 
        os.environ['SERVER_SOFTWARE'].startswith('voila')):      
    # localhost:8866?foo=bar,baz will return {'foo': ['bar', 'baz']}
    query_string = os.environ['QUERY_STRING']
    URL = os.environ['VOILA_SERVER_URL']
    parameters = parse_qs(query_string)
    display("query string parameters:", parameters)
else:
    # We're not running in voila, default to no parameters.
    URL = './'
    parameters = {}

In [10]:
from ipyleaflet import basemap_to_tiles, basemaps
import ipywidgets
import time

import dataportal_widgets
import output_options
import interactive_options

output_handler = output_options.OutputWidgetHandler()
stderr_handler = logging.StreamHandler(stream=sys.stderr)
logging.getLogger().addHandler(output_handler)
logging.getLogger().addHandler(stderr_handler)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('pygeoprocessing').setLevel(logging.DEBUG)

chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

if 'initial_layer' in parameters:
    initial_layer = parameters['initial_layer']
else:
    # just assume SRTM if the user hasn't specified a layer.
    initial_layer = "srtm-v3-1s"

m = dataportal_widgets.DataportalMap(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
    initial_layer=initial_layer
)

epsg_input = output_options.EPSG_INPUT
epsg_label = output_options.EPSG_LABEL
pixel_sizes_label = output_options.PIXEL_SIZES_LABEL


def _detect_srs_name(change):
    try:
        if isinstance(change['new'], str):
            epsg_code = int(change['new'])
        elif isinstance(change['new'], dict):
            epsg_code = int(change['new']['value'])
        else:
            raise Exception(change)
    except (ValueError, TypeError):
        epsg_code = None
    except KeyError:
        # no new value was set; skip
        return
        
    label, pixel_size_text = output_options.parse_epsg_code(epsg_code)
        
    epsg_label.value = label
    pixel_sizes_label.value = pixel_size_text
    
epsg_input.observe(_detect_srs_name, 'value')

pixel_size_x_input = output_options.PIXEL_SIZE_X_INPUT
pixel_size_y_input = output_options.PIXEL_SIZE_Y_INPUT 

reproject_cb = ipywidgets.Checkbox(
    value=False,
    description='Reproject to a local projection',
    layout={'width': 'max-content'},
    disabled=False,
    indent=False
)

def _enable_projection_info(value):
    epsg_input.disabled = not value['new']
    pixel_size_x_input.disabled = not value['new']
    pixel_size_y_input.disabled = not value['new']

reproject_cb.observe(_enable_projection_info, 'value')

dataset_label = ipywidgets.Label("")
input_descriptions_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Source layer</b>"),
    ipywidgets.HBox([ipywidgets.Label(value="Name:"), dataset_label]),
    ipywidgets.HBox([ipywidgets.Label(value="Projection: EPSG:4326")]),
    ipywidgets.HBox([ipywidgets.Label(value="Pixel size:"),
                     ipywidgets.Label(value="(1, 1)")])
])

output_params_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Output options</b>"),
    reproject_cb,
    ipywidgets.HBox([epsg_input, epsg_label]),
    ipywidgets.HBox(
        [pixel_size_x_input, pixel_size_y_input, pixel_sizes_label]),
])

logging_box = ipywidgets.VBox([
    ipywidgets.HTML("<b>Logging</b>"),
    output_handler.out,
])

submit = ipywidgets.Button(
    description='Clip raster',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clip the selected raster by the selected bounding box',
    icon='crop' # (FontAwesome names without the `fa-` prefix)
)
download_link = ipywidgets.HTML()
submit_hbox = ipywidgets.HBox([submit, download_link])
def _submit(button):
    button.disabled=True
    try:
        dataset_key = m.selected_map()
        timestamp = time.time()
        target_path = f'{dataset_key}-{timestamp}.tif'
        interactive_options.compute(
            f"/vsicurl/{dataportal_widgets.get_url(m.selected_map())}",
            m.selected_bbox(),
            epsg_input.value,
            target_path,
            (float(pixel_size_x_input.value), float(pixel_size_y_input.value)),
        )
        LOGGER.info("Making file public for 7 days")
        download_url = interactive_options.upload_to_gcs(target_path, layer_slug)
        LOGGER.info("Upload complete.")
        LOGGER.info(f"File can be downloaded from {download_url}")
        download_link.value = f"<a href={download_url}>Download {os.path.basename(download_url)}</a>"
    except Exception:
        LOGGER.exception("Compute/upload failed")
    finally:
        # always re-enable the button afterwards
        if os.path.exists(target_path):
            try:
                LOGGER.info(f"Cleaning up warped file {target_path}")
                os.remove(target_path)
            except Exception:
                LOGGER.exception(f"Could not remove {target_path}; skipping")
        button.disabled=False
    
submit.on_click(_submit)

display(m)
display(input_descriptions_box)
display(output_params_box)
display(submit_hbox)
display(logging_box)


DataportalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found


In [ ]:
pixel_size_x_input = output_options.PIXEL_SIZE_X_INPUT


reproject_cb = ipywidgets.Checkbox(
    value=False,
    description='Reproject to a local projection',
    layout={'width': 'max-content'},
    disabled=False,
    indent=False
)

def _doit(value):
    pixel_size_x_input.disabled = not value['new']

reproject_cb.observe(_doit, 'value')
display(pixel_size_x_input)
display(reproject_cb)

In [ ]:
# Remaining TODOs for this application:
# - [ ] expand this to operate on various layers available
# - [ ] expand this to write out rasters to a new session (grouped by uuid4()) on a bucket (maybe GDAL can directly write out to the bucket?)
# - [x] get the URL when running in Voila mode
# - [x] show the bounding box of the selected AOI polygon
# - [x] Allow the user to define their target EPSG code
# - [x] Use the user's target EPSG code in warping
# - [x] If the EPSG code represents a projected coord system (!osr.IsGeographic()), provide the pixel size
# - [x] Use osr.SpatialReference().GetName() to display the name of the coordinate system for user reference (unknown if unknown)